In [1]:
from brian2 import *
#import numpy as np
# from matplotlib import pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [3]:
a = arange(1, 10)
b = arange(2, 11)
dot(a, b)

330

Excitatory Neurons:
$$ \tau_m^+ \frac{\phi^P(\vec r, t+\Delta t)  - \phi^p(\vec r, t)}{\Delta t} + \phi^P(\vec r, t) = \Sigma_{P', \vec r'} w^+(\vec r - \vec r' - \xi \vec e^P)s^{P'}(\vec r', t - \tau_s^{++}) + \Sigma_{\vec r'}w^-(\vec r - \vec r')s^-(\vec r', t - \tau_s^-) + a^+(\vec r)[1 + \alpha E^P \bullet V(t)] + \zeta^P(\vec r, t)  $$

Inhibitory Neurons:
$$ \tau_m^+ \frac{\phi^-(\vec r, t+\Delta t)  - \phi^-(\vec r, t)}{\Delta t} + \phi^-(\vec r, t) = \Sigma_{P', \vec r'} w^+(\vec r - \vec r' - \xi \vec e^P)s^{P'}(\vec r', t - \tau_s^{++}) + a^-(t) + \zeta^-(\vec r, t)  $$

Synaptic connectivity from excitatory to all neurons
$$ w^+(\vec r) = \begin{cases}
                w_{mag} ^+{\frac{1 + cos(\pi r/r_{w^+})}{2}}& \text{; } r < rw+\\
                0          & \text{; } r\geq rw+
         \end{cases}
                $$
Synaptic connectivity from inhibitory to excitatory neurons
$$ w^-(\vec r) = \begin{cases}
                -w_{mag} ^-{\frac{1 + cos(\pi r/r_{w^+})}{2}}& \text{; } r < 2rw-\\
                0          & \text{; } r\geq 2rw-
         \end{cases}
                $$

$$ a^+(\vec r) = \begin{cases}
                -a_{min}^+ + (a^+_{max} - a^+_{min})^{\frac{1 + cos(\pi \rho /\rho_{a^+})}{2}}& \text{; } \rho < \rho a+\\
                a_{min}^+          & \text{; } \rho \geq \rho a+.
         \end{cases}
                $$

$$ a^-(t) = a_{mag}^- - a_{th}^-cos(t\pi ft + \psi_0)  $$

$$ \rho = \frac{\sqrt{(x - \frac{n+1/2}{2}^2 + (y - \frac{n+1}{2}^2)}}{n/2}

In [4]:
start_scope() # creat a new scope

# Parameters
N = 232 * 232 # Neurons per population

tau_m_plus = 40*ms # Exc. membrane time constant
tau_m_minus = 20*ms # Inh. membrane time constant
tau_s_plus_plus = 5*ms # Exc.-to-exc. synaptic delay
tau_s_minus_plus = 2*ms # Exc.-to-inh. synaptic delay
tau_s_minus = 2*ms # Inh. synaptic delay
a_max_plus = 2 # Exc. drive maximum
a_min_plus = 0.8 # Exc. drive minimum
rho_a_plus = 1.2 # Exc. drive scaled speed
a_mag_minus = 0.72 # Inh. drive magnitude
a_th_minus = 0.2 # Inh. drive theta amplitude
f = 8 # Inh. drive theta frequency
w_mag_plus = 0.2 # Exc. synaptic strength
r_w_plus = 6 # Exc. synaptic spread
w_mag_minus = 2.8 # Inh. synaptic strength
r_w_minus = 12 # Inh. synaptic distance
xi = 3 # Exc. synaptic shift
alpha = 0.25*second/metre # Exc. velocity gain
var_zeta_P = 0.002**2 # Exc. noise magnitude
var_zeta_I = 0.002**2 # Inh. noise magnitude


duration = 1000*ms


SyntaxError: invalid syntax (<ipython-input-4-20f4a5c5484c>, line 5)

In [ ]:
eqns_exc = '''
dv/dt = -v/tau_m_plus  + xi + a_plus*(1 + alpha*dot([xdir, ydir], ))

x: metre
y: metre

'''

eqns_inh = '''

dV/dt = -(v - a_minus)/tau_m_minus + xi

a_minus = a_mag_minus - a_th_minus*cos(2*pi*f*t): 1
'''


reset = '''

v = 0

'''

threshold = '''
v>1
'''

In [ ]:
# Neural Populations

## North
P_n = NeuronGroup(N, eqns, threshold='', reset=reset, method='heun')
P_n.x = 'i % 232' # will assign x-values from 0 to 231 repeatedly
P_n.y = 'i//232' # will assign y-value 0 to first 232 cells and so on
P_n.dir = '[0, 1]' # North
P_n.rho = 'sqrt((x - ((n+1)/2))**2 + (y - ((n+1)/2))**2)/(N/2)'
P_n.a_plus = 'where(rho < rho_a_plus, a_min_plus + (a_max_plus - a_min_plus)*(1 - cos(pi*rho/rho_a_plus)), a_min_plus)'


## South
P_s = NeuronGroup(N, eqns, threshold='', reset=reset, method='')
P_s.x = 'i % 232' # will assign x-values from 0 to 231 repeatedly
P_s.y = 'i//232' # will assign y-value 0 to first 232 cells and so on
P_s.dir = '[0, -1]' # South
P_s.rho = 'sqrt((x - ((n+1)/2))**2 + (y - ((n+1)/2))**2)/(N/2)'
P_s.a_plus = 'where(rho < rho_a_plus, a_min_plus + (a_max_plus - a_min_plus)*(1 - cos(pi*rho/rho_a_plus)), a_min_plus)'


## East
P_e = NeuronGroup(N, )
P_e.x = 'i % 232' # will assign x-values from 0 to 231 repeatedly
P_e.y = 'i//232' # will assign y-value 0 to first 232 cells and so on
P_e.dir = '[1, 0]' # East
P_e.rho = 'sqrt((x - ((n+1)/2))**2 + (y - ((n+1)/2))**2)/(N/2)'
P_e.a_plus = 'where(rho < rho_a_plus, a_min_plus + (a_max_plus - a_min_plus)*(1 - cos(pi*rho/rho_a_plus)), a_min_plus)'

## West
P_w = NeuronGroup(N, )
P_w.x = 'i % 232' # will assign x-values from 0 to 231 repeatedly
P_w.y = 'i//232' # will assign y-value 0 to first 232 cells and so on
P_w.dir = '[-1, 0]' # West
P_w.rho = 'sqrt((x - ((n+1)/2))**2 + (y - ((n+1)/2))**2)/(N/2)'
P_w.a_plus = 'where(rho < rho_a_plus, a_min_plus + (a_max_plus - a_min_plus)*(1 - cos(pi*rho/rho_a_plus)), a_min_plus)'

## Inhibitory
P_i = NeuronGroup(N, )
P_i.x = 'i % 232' # will assign x-values from 0 to 231 repeatedly
P_i.y = 'i//232' # will assign y-value 0 to first 232 cells and so on
P_i.a_minus = 'a_mag_minus - a_th_minus*cos(2*pi*f'


M_n = SpikeMonitor(P_n)
M_s = SpikeMonitor(P_s)
M_e = SpikeMonitor(P_e)
M_w = SpikeMonitor(P_w)



In [ ]:
S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'

S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'

S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'

S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'

S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'

S = Synapses(G, G, 'w: 1')
S.connect(condition='i!=j')
S.w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - zeta*xdir_pre)**2 + (y_post - y_pre - zeta*ydir_pre)**2)))'


S = [] # to store  the 25 synapse classes
exc_populations = [P_n, P_s, P_e, P_w]
all_populations = [P_n, P_s, P_e, P_w, P_i]
index = 0

# Set connections from excitatory to excitatory populations:
for src in exc_populations:
    for trg in exc_populations:
        S.append(Synapses(src, trg, 'w: 1', on_pre='v_post += w'))
        if src == trg: # connection within the population     
            S[index].connect(condition='i!=j') # if connection within population, don't connect neurons to themselves
        else:
            S[index].connect() # if connections are between two populations, connect all neurons
        S[index].w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - xi*xdir_pre)**2 + (y_post - y_pre - xi*ydir_pre)**2))))'
        S.delay = 'tau_s_plus_plus'
        index += 1

# Set connections from excitatory to inhibitory population:
for i in exc_populations:
    S.append(Synapses(i, P_i, 'w:1', on_pre='v_post += w'))
    S[index].connect()
    S[index].w = 'w_plus_mag**(1 + cos(pi*sqrt((x_post - x_pre - xi*xdir_pre)**2 + (y_post - y_pre - xi*ydir_pre)**2))))'
    S.delay = 'tau_s_minux_plus'
    index += 1    

# Set connections from inhibitory to excitatory neurons:
for i in exc_populations:
    S.append(Synapses(P_i, i, 'w:1', on_pre='v_post += w'))
    S[index].connect()
    S[index].w = 'w_minus_mag**(1 + cos(pi*sqrt((x_post - x_pre - xi*xdir_pre)**2 + (y_post - y_pre - xi*ydir_pre)**2)))'
    S.delay = 'tau_s_minus'
    index += 1

# The inhibitory population doesn't have recurrent connections within itself

run(duration)




Where to get the velocity inputs from?